<a href="https://colab.research.google.com/github/tsenga2/keio-quant-macro/blob/main/GrowthAccounting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#steo1
!pip install pandas openpyxl
!pip install jinja2


#step2
import pandas as pd

# データファイルのパスを指定（適切なファイルパスに置き換えてください）
file_path = 'C:/Users/shiba/OneDrive/ドキュメント/定量的マクロ/pwt1001.xlsx'  # 例: '/content/drive/MyDrive/pwt1001.xlsx'

# データを読み込む
pwt_data = pd.read_excel(file_path, sheet_name='Data')

# 必要なカラムを選択する
columns_needed = ['country', 'year', 'rgdpna', 'rkna', 'emp', 'avh', 'hc', 'labsh']
pwt_data = pwt_data[columns_needed]

# 分析期間のデータをフィルタリング
pwt_data = pwt_data[(pwt_data['year'] >= 1990) & (pwt_data['year'] <= 2019)]

# 対象の国リストを作成
target_countries = ['Australia', 'Austria', 'Belgium', 'Canada', 'Denmark', 'Finland', 'France', 
                    'Germany', 'Greece', 'Iceland', 'Ireland', 'Italy', 'Japan', 'Netherlands', 
                    'New Zealand', 'Norway', 'Portugal', 'Spain', 'Sweden', 'Switzerland', 
                    'United Kingdom', 'United States']

# 対象国のみのデータにフィルタリング
pwt_data = pwt_data[pwt_data['country'].isin(target_countries)]

# データの確認
pwt_data.head()





#step3
# 国ごとのデータフレームに分割
countries = pwt_data['country'].unique()
country_data = {country: pwt_data[pwt_data['country'] == country] for country in countries}

results = []

for country, data in country_data.items():
    data = data.sort_values('year')
    
    # 変数の計算
    data['rgdpna_growth'] = data['rgdpna'].pct_change() * 100
    data['capital_deepening'] = (data['rkna'] / (data['emp'] * data['avh'])).pct_change() * 100
    data['labor_quality'] = data['hc'].pct_change() * 100
    
    # 労働シェアと資本シェアの計算
    labor_share = data['labsh'].mean()  # 平均労働シェア
    capital_share = 1 - labor_share
    
    # TFPの計算
    data['tfp'] = data['rgdpna'] / ((data['rkna'] ** capital_share) * ((data['emp'] * data['avh'] * data['hc']) ** labor_share))
    data['tfp_growth'] = data['tfp'].pct_change() * 100
    
    # 各国の平均を計算
    avg_growth_rate = data['rgdpna_growth'].mean()
    avg_tfp_growth = data['tfp_growth'].mean()
    avg_capital_deepening = data['capital_deepening'].mean()
    
    results.append([country, avg_growth_rate, avg_tfp_growth, avg_capital_deepening, labor_share, capital_share])

# 結果をデータフレームに変換
results_df = pd.DataFrame(results, columns=['Country', 'Growth Rate', 'TFP Growth', 'Capital Deepening', 'TFP Share', 'Capital Share'])

# 結果を表示
print(results_df)


#step4
# 表のスタイル設定
styled_results = results_df.style.format({
    'Growth Rate': "{:.2f}",
    'TFP Growth': "{:.2f}",
    'Capital Deepening': "{:.2f}",
    'TFP Share': "{:.2f}",
    'Capital Share': "{:.2f}"
})

# 表の表示
styled_results


           Country  Growth Rate  TFP Growth  Capital Deepening  TFP Share  \
0        Australia     3.026192    0.589861           2.187813   0.595819   
1          Austria     1.908098    0.242078           2.394917   0.592833   
2          Belgium     1.811803   -0.035070           2.266523   0.616044   
3           Canada     2.299039    0.185398           2.183898   0.665831   
4      Switzerland     1.661622    0.154914           2.366808   0.680107   
5          Germany     1.498944    0.589721           2.163243   0.637198   
6          Denmark     1.803654    0.348770           2.344613   0.636907   
7            Spain     2.065071   -0.276776           2.062333   0.606252   
8          Finland     1.773624    0.590581           2.037436   0.594583   
9           France     1.574034    0.158596           2.009157   0.618732   
10  United Kingdom     2.015325    0.269855           1.984615   0.582796   
11          Greece     0.943917   -0.348070           1.800483   0.519861   

,Country,Growth Rate,TFP Growth,Capital Deepening,TFP Share,Capital Share
0,Australia,3.03,0.59,2.19,0.60,0.40
1,Austria,1.91,0.24,2.39,0.59,0.41
2,Belgium,1.81,-0.04,2.27,0.62,0.38
3,Canada,2.30,0.19,2.18,0.67,0.33
4,Switzerland,1.66,0.15,2.37,0.68,0.32
5,Germany,1.50,0.59,2.16,0.64,0.36
6,Denmark,1.80,0.35,2.34,0.64,0.36
7,Spain,2.07,-0.28,2.06,0.61,0.39
8,Finland,1.77,0.59,2.04,0.59,0.41
9,France,1.57,0.16,2.01,0.62,0.38
